In [1]:
from transformers import AutoTokenizer
cp = "Tables"
tok = AutoTokenizer.from_pretrained(cp)

In [2]:
class searchResult:
    def __init__(self):
        self.start = 0
        self.end = 0
        self.txt = ''
        self.extract = ''

In [104]:
class NLPInvoker:
    def __init__(self):
        self.gCmds = []
        
    def split4(self,inputs):
        result = []
        current = []
        a = inputs[0][1:]
        for e in a:
            if e != 4 : 
                current.append(e)
            else:
                result.append(current)
                current = []
        result.append(current)
        return result
        
    def _search(self,inputs,splits,idx):
        result = []
        current = []
        deb = -1
        fin = -1
        i = 0
        cnt = 0
        cntLoss = 0
        # print(inputs)
        toSearch = splits[idx]
        # print(toSearch)
        b = inputs[1]
        for e in b:
            # print(e,cnt)
            if deb == -1 :
                if e==toSearch[cnt]:
                    cnt = cnt + 1
                    deb = i
                    cntLoss = 0 
                    if len(toSearch) == 1:
                        fin = i+1
                        cnt = 0
                        cntLoss = 0
                        #print("found")
                        break
            else:
                if e==toSearch[cnt]:
                    cnt = cnt + 1
                    cntLoss = 0 
                    if cnt ==len(toSearch):
                        fin = i+1
                        #print("found")
                        break
                else:
                    cntLoss = cntLoss + 1
                    if cntLoss>2:
                        deb = -1
                        cnt = 0
            i = i +1
        return [deb,fin]
    
    def search(self,inputs):
        #print(inputs)
        split = self.split4(inputs)
        result = []
        first = True
        for i in range(len(split)):
            lst = self._search(inputs,split,i)
            if i != len(split)-1:
                result.append(searchResult())
                result[i].start = lst[1]
            if i !=0:
                result[i-1].end = lst[0]
        for i in range(len(split)-1):
            result[i].extract = tok.decode(inputs[1][result[i].start:result[i].end]).replace(" ##","")
            result[i].txt = tok.decode(split[i]).replace(" ##","")
    
        found = True
        for r in result:
            if r.start == -1:
                found = False
                break
            if r.end == -1:
                found = False
                break
           
        return found, result   
    
    def addCmd(self, nomCommande, question, where, params):
        self.gCmds.append([where,nomCommande,tok(question.replace("?","[SEP]")).input_ids[1:], params])
        print(tok.decode(tok(question).input_ids[1:]))
    
    def Comment(self,Ordre):
        encoding = tok.encode(Ordre)
        #print(encoding)
        #print(tok.decode(encoding, True))
        encoding[0] = encoding[1]
        encoding[1] = 67
        encoding.insert(0,17475)
        encoding[len(encoding)-1] = 3
        if (171 in encoding):
            encoding.remove(171)
        #print(encoding)
        return tok.decode(encoding, True).replace(" ##","").replace(" ' ","'")

    def execute(self,ordre):
        how = self.Comment(ordre)+" [SEP]"
        print(how)
        for c in self.gCmds:
            #print(c[2])
            f,res = self.search([c[2],tok(how).input_ids])
            if f:
                ch = ""
                for r in res:
                    ch = ch + ", "+ r.extract
                print("=>", ch )
                
                return c,res

        return how.replace("[SEP]","?")

In [105]:
lia = NLPInvoker()
#lia.addCmd("constantes", "NewTemplate", "appliquer le template suivant [MASK] [SEP]", "general")
#lia.addCmd("constantes", "explain", "donner comme explication [MASK] [SEP]", "general")
#lia.addCmd("constantes", "code", "appliquer le template [MASK] [SEP].","applyTemplate")
#lia.addCmd("constantes", "code", "ajouter la constante [MASK] ayant pour valeur [MASK] [SEP]", "addConst")
#lia.addCmd("constantes", "code", "ajouter la propriété [MASK] de type [MASK] [SEP]", "addprop")
#lia.addCmd("constantes", "code", "créer une unité ayant pour nom [MASK] [SEP]", "createUnit")
lia.gCmds

[]

In [106]:
import yaml,sys
d=yaml.safe_load(open('Templates/Delphi.yaml', 'r'))

for c in d["Commands"]:
    e = c["Command"]
    print(e["Question"])
    lia.addCmd("code", e["Question"], e["Ou"], e)


Comment créer une unité ayant pour nom [MASK] ?
creer une unite ayant pour nom [MASK] ?
Comment ajouter une constante [MASK] ayant pour valeur [MASK] ?
ajoute ##r une constant ##e [MASK] ayant pour valeur [MASK] ?
Comment ajouter une classe [MASK] ?
ajoute ##r une classe [MASK] ?
Comment ajouter une procédure [MASK] ?
ajoute ##r une pro ##ce ##dure [MASK] ?
Comment ajouter une fonction [MASK] qui retourne [MASK] ?
ajoute ##r une fonction [MASK] qui retour ##ne [MASK] ?


In [107]:
#lia.gCmds

In [108]:
input1="donne comme explication que tu es la pour aider au codage d'une dll d'api rest."
while input1 != "end":
    print(" LIA >",lia.execute("* "+input1))
    input1 = input("USER >")

comment donner comme explication que tu es la pour aider au codage d'une dll d'api rest [SEP]
 LIA > comment donner comme explication que tu es la pour aider au codage d'une dll d'api rest ?


USER > je voudrais ajouter une procédedure toto.pas.


comment jer voudrais ajouter une procededure toto . pas [SEP]
=> , toto . pas
 LIA > (['implementation', 'code', [17156, 67, 2244, 501, 155, 1148, 4, 3], {'Name': 'addProcedure', 'Question': 'Comment ajouter une procédure [MASK] ?', 'Params': [{'Param': {'Nom': 'NomProcedure', 'Libelle': 'nom de la procedure', 'Traitements': 'Variabilize', 'Questions': ['Quel est le nom de la procedure ?']}}], 'Ou': 'implementation', 'Pourquoi': "Permet d'effectuer une liste de traitements.", 'Template': 'procedure §NomProcedure§([Parametres]); const [Constantes] var [variables] begin [instructions] end;'}], [<__main__.searchResult object at 0x7f1c93319b70>])


USER > ajouter une procedure toto.pas


comment ajouterr une procedure toto . [SEP]
=> , toto .
 LIA > (['implementation', 'code', [17156, 67, 2244, 501, 155, 1148, 4, 3], {'Name': 'addProcedure', 'Question': 'Comment ajouter une procédure [MASK] ?', 'Params': [{'Param': {'Nom': 'NomProcedure', 'Libelle': 'nom de la procedure', 'Traitements': 'Variabilize', 'Questions': ['Quel est le nom de la procedure ?']}}], 'Ou': 'implementation', 'Pourquoi': "Permet d'effectuer une liste de traitements.", 'Template': 'procedure §NomProcedure§([Parametres]); const [Constantes] var [variables] begin [instructions] end;'}], [<__main__.searchResult object at 0x7f1c9335b550>])


USER > je vourdrais créer une unité ayant pour nom toto.pas.


comment jer vourdrais creer une unite ayant pour nom toto . pas [SEP]
=> , toto . pas
 LIA > (['Contenu', 'code', [13335, 2244, 1797, 4618, 225, 455, 4, 3], {'Name': 'addUnit', 'Question': 'Comment créer une unité ayant pour nom [MASK] ?', 'Params': [{'Param': {'Nom': 'NomUnite', 'Libelle': "nom de l'unité", 'Traitements': 'Variabilize', 'Questions': ["Quel est le nom de l'unité ?"]}}], 'Ou': 'Contenu', 'Pourquoi': 'Pour contenir le code source.', 'Template': 'Unit §NomUnite§\ninterface\nuses [UsesListInt]\nconst [ConstList]\ntype [TypeList]\nvar [GlobalVarList]\nimplementation\nuses [UsesListImp]\n[Implementation]\nend.'}], [<__main__.searchResult object at 0x7f1c9335b550>])


USER > je voudrais ajouter une procédure toto.pas.


comment jer voudrais ajouter une procedure toto . pas [SEP]
=> , toto . pas
 LIA > (['implementation', 'code', [17156, 67, 2244, 501, 155, 1148, 4, 3], {'Name': 'addProcedure', 'Question': 'Comment ajouter une procédure [MASK] ?', 'Params': [{'Param': {'Nom': 'NomProcedure', 'Libelle': 'nom de la procedure', 'Traitements': 'Variabilize', 'Questions': ['Quel est le nom de la procedure ?']}}], 'Ou': 'implementation', 'Pourquoi': "Permet d'effectuer une liste de traitements.", 'Template': 'procedure §NomProcedure§([Parametres]); const [Constantes] var [variables] begin [instructions] end;'}], [<__main__.searchResult object at 0x7f1c93319b70>])


USER > end


In [103]:
lia.execute("je voudrais ajouter la constante Nom d'utiliseur ayant pour valeur toto [SEP]")

comment vourdrais ajouter la constante nom d'utiliseur ayant pour valeur toto [SEP]


"comment vourdrais ajouter la constante nom d'utiliseur ayant pour valeur toto ?"

In [19]:
lia.execute("comment ajouter la propriété Nom d'utiliseur de type string [SEP]")

IndexError: list index out of range

In [118]:
lia.search([tok("ajouter la propriété [MASK] de type [MASK] [SEP]").input_ids,tok("je voudrais ajouter la propriété Nom d'utiliseur de type string [SEP]").input_ids])


(True,
  <__main__.searchResult at 0x7f66080f7b80>])

In [119]:
lia.Comment("* ajoute une propriété nom.")

'comment ajouter une propriete nom'

In [120]:
print(lia.Comment("* ajoute la propriété nom d'utilsateur de type Integer [SEP]"))
#lia.execute(lia.Comment("* ajoute la propriété nom d'utilsateur de type Integer [SEP]"))

comment ajouter la propriete nom d'utilsateur de type integer


In [121]:
lia.execute(lia.Comment("* ajoute la propriété nom d'utilsateur de type Integer [SEP]")+" [SEP]")

"comment ajouterr la propriete nom d'utilsateur de type integer ?"

In [122]:
lia.execute("* ajoute la constante fichier Ini ayant pour valeur le nom de la DLL.")

'comment ajouter la constante fichier ini ayant pour valeur le nom de la dll ?'

In [123]:
lia.execute("* ajoute la variable nom d'utilisateur de type string.")

"comment ajouter la variable nom d'utilisateur de type string ?"

In [124]:
def q(question):
    return lia.execute("* "+question)

In [125]:
q("ajoute la constante password ayant pour valeur toto.")

'comment ajouter la constante password ayant pour valeur toto ?'

In [126]:
q("cree une unitée ayant pour nom toto.")

'comment creer une unitee ayant pour nom toto ?'

In [127]:
q("cree un singleton qui a pour nom le nom de la dll suivi de Contexte.")

'comment creer un singleton qui a pour nom le nom de la dll suivi de contexte ?'

In [128]:
q("ajoute la constante titi ayant pour valeur le nom de la dll suivi de Contexte.")

'comment ajouter la constante titi ayant pour valeur le nom de la dll suivi de contexte ?'

In [129]:
q("cree une nouvelle unitée ayant pour nom toto.")

'comment creer une nouvelle unitee ayant pour nom toto ?'

In [130]:
q("cree une unitée ayant pour nom toto.")

'comment creer une unitee ayant pour nom toto ?'

In [131]:
a = q("""applique le template suivant : 
|*|C|NomConstante| = |Valeur|;
""")
tok("""applique le template suivant : 
|*|C|NomConstante| = |Valeur|;
""")

{'input_ids': [7408, 467, 1465, 2453, 73, 8190, 408, 0, 0, 0, 0, 30, 0, 455, 1582, 2634, 0, 22, 0, 195, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [132]:
q("applique le template addConst.")

'comment appliquer le template addconst ?'

In [133]:
q("donne comme explication que tu n'est pas programmée pour ça, que tu es désolée de ne pas pouvoir répondre.")

"comment donner comme explication que tu n'est pas programmee pour ca , que tu es desolee de ne pas pouvoir repondre ?"

In [134]:
q("demande a quoi sert le paramètre [MASK].")

'comment demander a quoi sert le parametre ?'

In [135]:
q("si tu ne sais pas comment faire, alors, donne comme explication que tu es désolée de ne pas pouvoir répondre.")

'comment sir tu ne sais pas comment faire , alors , donne comme explication que tu es desolee de ne pas pouvoir repondre ?'

In [136]:
q("""il faudrait appliquer le template suivant : 
fjsqdlkfjsdqjflksdqf
""")

'comment ilr faudrait appliquer le template suivant fjsqdlkfjsdqjflksdq ?'

In [137]:
q("""je voudrais que tu appliques le template suivant : 
fjsqdlkfjsdqjflksdqf
""")

'comment jer voudrais que tu appliques le template suivant fjsqdlkfjsdqjflksdq ?'

In [140]:
input1="donne comme explication que tu es la pour aider au codage d'une dll d'api rest."
while input1 != "end":
    print(" LIA >",q(input1))
    input1 = input("USER >")


 LIA > comment donner comme explication que tu es la pour aider au codage d'une dll d'api rest ?


USER > ajoute une unitée.


 LIA > comment ajouter une unitee ?


USER > Crée un nouveau fichier vide.


 LIA > comment creer un nouveau fichier vide ?


USER > Crée un nouveau fichier vide ayant pour nom toto.pas.


 LIA > comment creer un nouveau fichier vide ayant pour nom toto . pas ?


KeyboardInterrupt: Interrupted by user